In [1]:
import os
import sys
import numpy as np
import json



In [3]:
mac_file = '../devices.txt'
mac_dic = {}
with open(mac_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        tmp_mac, tmp_device = line[:-1].split(' ')
        if len(tmp_mac) != 17:
            mac_split = tmp_mac.split(':')
            for i in range(len(mac_split)):
                if len(mac_split[i]) != 2:
                    mac_split[i]='0'+mac_split[i]
            tmp_mac = ':'.join(mac_split)
        mac_dic[tmp_device] = tmp_mac
# print(mac_dic)



In [5]:


device_category = {
    'home auto': ["amazon-plug", "bulb1", "dlink-mov", "gosund-bulb1", "govee-led1", "keyco-air", "magichome-strip", "meross-dooropener", "nest-tstat", "philips-bulb", "ring-chime1", "smartlife-bulb", "t-wemo-plug", "thermopro-sensor", "tplink-bulb", "tplink-plug"],
    'camera': ["dlink-camera", "icsee-doorbell", "lefun-cam-wired", "microseven-camera", "ring-camera", "ring-doorbell", "tuya-camera", "ubell-doorbell", "wansview-cam-wired", "wyze-cam", "yi-camera"],
    'smart speaker': ["echodot", "echodot3a", "echodot3c", "echodot4b", "echoflex1", "echoplus", "echoshow5", "echospot", "google-home-mini", "google-nest-mini1", "homepod", "homepod-mini1"],
    'hub': ["aqara-hub", "ikea-hub", "smartthings-hub", "switchbot-hub", "t-philips-hub", "wink-hub2"],
    'appliance': ["brewer", "dishwasher", "fridge", "ikettle", "microwave", "sousvide"]
}

In [7]:
periodic_file_dir = '../event_inference/period_detection/freq_period/fingerprints'

periodic_model_per_device = {}
dns_per_device = {}
ntp_per_devices = {}
for device_file in os.listdir(periodic_file_dir):
    if not device_file.endswith('.txt'):
        continue
    device_name = device_file.split('.')[0]
    # print(device_name)
    with open(os.path.join(periodic_file_dir, device_file), 'r') as f:
        lines = f.readlines()
        count = 0
        for line in lines:
            line = line.strip()
            # print(line)
            prot, domain, period = line.split()[0:3]
            if  domain in mac_dic or domain in ['multicast', 'ff:ff:ff:ff:ff:ff'] or domain.startswith('192.168.10') : # prot == 'DHCP' or
                continue
            count += 1
            if prot == 'DNS':
                if not 'neu' in domain:
                    dns_per_device[domain] = dns_per_device.get(domain, [])
                    dns_per_device[domain].append(device_name)
            if prot == 'NTP':
                ntp_domain = '.'.join(domain.split('.')[-2:])
                ntp_per_devices[ntp_domain] = ntp_per_devices.get(ntp_domain, [])
                ntp_per_devices[ntp_domain].append(device_name)
        periodic_model_per_device[device_name]  = count
print(dns_per_device)
print('--NTP--', len(ntp_per_devices))
# print(ntp_per_devices)
# print(json.dumps(ntp_per_devices, indent=4))
# print(periodic_model_per_device)
print(np.average(list(periodic_model_per_device.values())))
print(np.sum(list(periodic_model_per_device.values())))
print(np.median(list(periodic_model_per_device.values())))

print(json.dumps(periodic_model_per_device, indent=4))

periodic_model_per_category = {}
for category in device_category:
    periodic_model_per_category[category] = []

for dev in periodic_model_per_device:
    for category in device_category:
        if dev in device_category[category]:
            
            periodic_model_per_category[category].append(periodic_model_per_device[dev])
            
for cat in periodic_model_per_category:
    periodic_model_per_category[cat] = sorted(periodic_model_per_category[cat])
print(json.dumps(periodic_model_per_category, indent=4))

for cat in periodic_model_per_category:
    periodic_model_per_category[cat] = [np.sum(periodic_model_per_category[cat]) / len(periodic_model_per_category[cat]), np.std(periodic_model_per_category[cat])]
print(json.dumps(periodic_model_per_category, indent=4))

{'dns.google': ['icsee-doorbell', 'fridge', 'wyze-cam', 'ring-chime1', 'google-home-mini', 'google-nest-mini1']}
--NTP-- 17
9.26530612244898
454
5.0
{
    "ikea-hub": 6,
    "icsee-doorbell": 10,
    "sousvide": 2,
    "ring-doorbell": 4,
    "tplink-plug": 5,
    "thermopro-sensor": 3,
    "echoplus": 23,
    "wink-hub2": 7,
    "fridge": 22,
    "t-wemo-plug": 4,
    "dlink-mov": 4,
    "lefun-cam-wired": 3,
    "echodot4b": 19,
    "homepod-mini1": 28,
    "microseven-camera": 4,
    "ikettle": 2,
    "nest-tstat": 8,
    "smartthings-hub": 4,
    "philips-bulb": 2,
    "dlink-camera": 3,
    "wyze-cam": 10,
    "t-philips-hub": 15,
    "ring-chime1": 5,
    "keyco-air": 3,
    "ring-camera": 6,
    "brewer": 2,
    "microwave": 4,
    "switchbot-hub": 2,
    "meross-dooropener": 5,
    "echoshow5": 32,
    "echoflex1": 16,
    "gosund-bulb1": 2,
    "bulb1": 2,
    "wansview-cam-wired": 6,
    "google-home-mini": 23,
    "govee-led1": 5,
    "ubell-doorbell": 9,
    "homepod": 26,
